In [35]:
import matplotlib.pyplot as plt
from itertools import product
import pennylane as qml
import pennylane.numpy as np
import pickle

num_bits = 6
NUM_LAYER = 28
with open('parameter 4-10 2.pkl', 'rb') as f:
    record = pickle.load(f)
    para, paraG = record[-1]

In [36]:
#%% import
import pennylane.numpy as np
import pennylane as qml
import pickle, copy
from DataGenerator import SwapSymmetry
with open('data_symmetry5.pkl', 'rb') as f:
        [num_bits, train_data, test_data, symmetry] = pickle.load(f)


dev = qml.device('default.mixed', wires=num_bits)

#%% Ansatz
def ansatz(parameters, num_layers):
    para_arr = [parameters[i:i+num_bits] for i in range(0, len(parameters), num_bits)]
    para_arr.reverse()

    qml.broadcast(qml.RZ, wires=range(num_bits), 
                  pattern="single", parameters=para_arr.pop() )
    for i in range(num_layers):
        pattern = "double" if i%2 == 0 else "double_odd"
        layer(num_bits, para_arr.pop(), pattern)
    qml.broadcast(qml.RX, wires=range(num_bits), 
                  pattern="single", parameters=para_arr.pop() )
    
@qml.qnode(dev, diff_method="backprop", interface="torch")
def circuit(x, parameters, num_layers):
    # x = np.append(x,x)
    # x = np.append(x,[0])
    qml.broadcast(qml.RY, wires=range(num_bits), 
                  pattern="single", parameters=x )
    ansatz(parameters, num_layers)

    return qml.expval(qml.PauliZ(num_bits-1))

def layer(num_bits, parameters, pattern):
    qml.broadcast(qml.RY, wires=range(num_bits), 
                  pattern="single", parameters=parameters )
    qml.broadcast(qml.CNOT, wires=range(num_bits), pattern=pattern)

# %% cost function

def cost(parameter, num_layers):
    # cst_values =  [(circuit(x, parameter, num_layers) + (-1)**label)**2
    #                 for x, label in train_data ]
    cst_values = []
    for x, label in train_data:
        cst_values.append(
            ( circuit(x, parameter, num_layers) + (-1)**label )**2
            )
    return sum(cst_values) / len(train_data)

@qml.qnode(dev, diff_method="backprop", interface="torch")
def U_circ(parameters, num_layer):
    ansatz(parameters, num_layer)
    return qml.expval(qml.Identity(0))

def costG(parameter, num_layer, lamd):
    cst = cost(parameter, num_layer)
    guid = g(parameter, num_layer)
    return cst + lamd * guid

def g(parameter, num_layer):
    U = qml.matrix(U_circ)(parameter, num_layer)
    return symmetry.symmetry_guidance(U)

def accuracy(parameter, num_layer):
    correct = 0
    for rho,label in test_data: 
        prediction = np.ceil(circuit(rho, parameter, num_layer))
        if np.abs(prediction - label) < 1 : correct += 1
    return correct / len(test_data)



from functools import partial


    
ETA = 0.04
LAMD = 3
MAX_ITER = 80
NUM_PARA = (NUM_LAYER + 2) * num_bits
opt = qml.AdamOptimizer(stepsize = ETA)

cst_lst = []
cstG_lst = []


cost_ = partial(cost, num_layers = NUM_LAYER)
costG_ = partial(costG, num_layer = NUM_LAYER, lamd = LAMD)
accuracy_ = partial(accuracy, num_layer = NUM_LAYER)



for it in range(MAX_ITER):

    para, cst = opt.step_and_cost(cost_, para)
    paraG, cstG = opt.step_and_cost(costG_, paraG)
    cst_lst.append(cst)
    cstG_lst.append(cstG/2)
    # ac, acG = accuracy_(para), accuracy_(paraG)
    record.append((para,paraG))

    if it % 4 == 0: 
        print( f"Iter: {it + 1 : 3d} | Cost: {cst : 0.4f} | CostG: {cstG : 0.4f} | accuracy: {accuracy_(para) : 3.2%} | accuracyG: {accuracy_(paraG) : 3.2%}" )



Iter:   1 | Cost:  0.0407 | CostG:  1.3546 | accuracy:  77.08% | accuracyG:  81.25%
Iter:   5 | Cost:  0.0373 | CostG:  1.3823 | accuracy:  81.25% | accuracyG:  80.21%
Iter:   9 | Cost:  0.0352 | CostG:  1.3548 | accuracy:  79.17% | accuracyG:  83.33%
Iter:  13 | Cost:  0.0355 | CostG:  1.3332 | accuracy:  82.29% | accuracyG:  82.29%
Iter:  17 | Cost:  0.0301 | CostG:  1.3216 | accuracy:  80.21% | accuracyG:  82.29%
Iter:  21 | Cost:  0.0297 | CostG:  1.3071 | accuracy:  81.25% | accuracyG:  82.29%
Iter:  25 | Cost:  0.0286 | CostG:  1.2925 | accuracy:  81.25% | accuracyG:  82.29%
Iter:  29 | Cost:  0.0283 | CostG:  1.2729 | accuracy:  82.29% | accuracyG:  82.29%
Iter:  33 | Cost:  0.0298 | CostG:  1.2427 | accuracy:  81.25% | accuracyG:  82.29%
Iter:  37 | Cost:  0.0294 | CostG:  1.2090 | accuracy:  81.25% | accuracyG:  80.21%
Iter:  41 | Cost:  0.0301 | CostG:  1.1763 | accuracy:  81.25% | accuracyG:  80.21%
Iter:  45 | Cost:  0.0312 | CostG:  1.1508 | accuracy:  81.25% | accuracyG: 

In [37]:

with open('parameter 4-10 2.pkl', 'wb') as f:
    pickle.dump(record, f)

In [38]:
len(record)

179